In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!curl -LO https://github.com/AakashKumarNain/CaptchaCracker/raw/master/captcha_images_v2.zip
!unzip -qq captcha_images_v2.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   159  100   159    0     0    203      0 --:--:-- --:--:-- --:--:--   203
100 8863k  100 8863k    0     0  5222k      0  0:00:01  0:00:01 --:--:-- 18.5M


In [16]:
import albumentations
import numpy as np
import pandas as pd

import PIL
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

import torch
import torch.nn as nn
from torch.nn import functional as F

import os
from glob import glob
from sklearn import preprocessing, model_selection, metrics


In [17]:
DATA_DIR = "./captcha_images_v2"
BATCH_SIZE = 8
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 75
NUM_WORKERS = 8
EPOCHS = 200
DEVICE = "cuda"

In [18]:
class CaptchaModel(nn.Module):
    def __init__(self, num_chars):
        super(CaptchaModel, self).__init__()
        self.conv_1 = nn.Conv2d(3, 128, kernel_size=(3, 6), padding=(1, 1))
        self.pool_1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv_2 = nn.Conv2d(128, 64, kernel_size=(3, 6), padding=(1, 1))
        self.pool_2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.linear_1 = nn.Linear(1152, 64)
        self.drop_1 = nn.Dropout(0.2)
        self.lstm = nn.GRU(64, 32, bidirectional=True, num_layers=2, dropout=0.25, batch_first=True)
        self.output = nn.Linear(64, num_chars + 1)

    def forward(self, images, targets=None):
        bs, _, _, _ = images.size()
        x = F.relu(self.conv_1(images))
        #print(x.shape)
        x = self.pool_1(x)
        #print(x.shape)
        x = F.relu(self.conv_2(x))
        #print(x.shape)
        x = self.pool_2(x)
        #print(x.shape)
        x = x.permute(0, 3, 1, 2)
        #print(x.shape)
        x = x.view(bs, x.size(1), -1)
        #print(x.shape)
        x = F.relu(self.linear_1(x))
        #print(x.shape)
        x = self.drop_1(x)
        #print(x.shape)
        x, _ = self.lstm(x)
#print(x.shape)
        x = self.output(x)
        #print(x.shape)
        x = x.permute(1, 0, 2)
        #print(x.shape)

        if targets is not None:
            log_probs = F.log_softmax(x, 2)
            input_lengths = torch.full(
                size=(bs,), fill_value=log_probs.size(0), dtype=torch.int32
            )
            target_lengths = torch.full(
                size=(bs,), fill_value=targets.size(1), dtype=torch.int32
            )
            loss = nn.CTCLoss(blank=0)(
                log_probs, targets, input_lengths, target_lengths
            )
            return x, loss

        return x, None


In [19]:
if __name__ == "__main__":
    cm = CaptchaModel(19)
    img = torch.rand((1, 3, 75, 300))
    x, _ = cm(img)
    

torch.Size([1, 128, 75, 297])
torch.Size([1, 128, 37, 148])
torch.Size([1, 64, 37, 145])
torch.Size([1, 64, 18, 72])
torch.Size([1, 72, 64, 18])
torch.Size([1, 72, 1152])
torch.Size([1, 72, 64])
torch.Size([1, 72, 64])
torch.Size([1, 72, 64])
torch.Size([1, 72, 20])
torch.Size([72, 1, 20])


In [20]:
class ClassificationDataset:
    def __init__(self, image_paths, targets, resize=None):
        # resize = (height, width)
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize

        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.225)
        self.aug = albumentations.Compose(
            [
                albumentations.Normalize(
                    mean, std, max_pixel_value=255.0, always_apply=True
                )
            ]
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, item):
        image = Image.open(self.image_paths[item]).convert("RGB")
        targets = self.targets[item]

        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]), resample=Image.BILINEAR
            )

        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented["image"]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        return {
            "images": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long),
        }


In [30]:

def run_training():
    DATA_DIR = "./captcha_images_v2/"
    BATCH_SIZE = 8
    IMAGE_WIDTH = 300
    IMAGE_HEIGHT = 75
    NUM_WORKERS = 8
    EPOCHS = 200
    DEVICE = "cuda"
    
    
    image_files = glob(os.path.join(DATA_DIR, "*.png"))
    targets_orig = [x.split("/")[-1][:-4] for x in image_files]
    targets = [[c for c in x] for x in targets_orig]
    targets_flat = [c for clist in targets for c in clist]

    lbl_enc = preprocessing.LabelEncoder()
    lbl_enc.fit(targets_flat)
    targets_enc = [lbl_enc.transform(x) for x in targets]
    targets_enc = np.array(targets_enc)
    targets_enc = targets_enc + 1

    (
        train_imgs,
        test_imgs,
        train_targets,
        test_targets,
        _,
        test_targets_orig,
    ) = model_selection.train_test_split(
        image_files, targets_enc, targets_orig, test_size=0.1, random_state=42
    )

    train_dataset = ClassificationDataset(
        image_paths=train_imgs,
        targets=train_targets,
        resize=(IMAGE_HEIGHT, IMAGE_WIDTH),
    )
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=True,
    )
    test_dataset = ClassificationDataset(
        image_paths=test_imgs,
        targets=test_targets,
        resize=(IMAGE_HEIGHT, IMAGE_WIDTH),
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size = BATCH_SIZE,
        num_workers = NUM_WORKERS,
        shuffle=False,
    )

    model = CaptchaModel(num_chars=len(lbl_enc.classes_))
    model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.8, patience=5, verbose=True
    )
    for epoch in range(EPOCHS):
        train_loss = train_fn(model, train_loader, optimizer)
        valid_preds, test_loss = eval_fn(model, test_loader)
        valid_captcha_preds = []
        for vp in valid_preds:
            current_preds = decode_predictions(vp, lbl_enc)
            valid_captcha_preds.extend(current_preds)
        combined = list(zip(test_targets_orig, valid_captcha_preds))
        print(combined[:10])
        test_dup_rem = [remove_duplicates(c) for c in test_targets_orig]
        accuracy = metrics.accuracy_score(test_dup_rem, valid_captcha_preds)
        print(
            f"Epoch={epoch}, Train Loss={train_loss}, Test Loss={test_loss} Accuracy={accuracy}"
        )
        scheduler.step(test_loss)

In [22]:
from tqdm import tqdm

def train_fn(model, data_loader, optimizer):
    model.train()
    fin_loss = 0
    tk0 = tqdm(data_loader, total=len(data_loader))
    for data in tk0:
        for key, value in data.items():
            data[key] = value.to("cuda")
        optimizer.zero_grad()
        _, loss = model(**data)
        loss.backward()
        optimizer.step()
        fin_loss += loss.item()
    return fin_loss / len(data_loader)


def eval_fn(model, data_loader):
    model.eval()
    fin_loss = 0
    fin_preds = []
    tk0 = tqdm(data_loader, total=len(data_loader))
    for data in tk0:
        for key, value in data.items():
            data[key] = value.to("cuda")
        batch_preds, loss = model(**data)
        fin_loss += loss.item()
        fin_preds.append(batch_preds)
    return fin_preds, fin_loss / len(data_loader)

In [23]:
def decode_predictions(preds, encoder):
    preds = preds.permute(1, 0, 2)
    preds = torch.softmax(preds, 2)
    preds = torch.argmax(preds, 2)
    preds = preds.detach().cpu().numpy()
    cap_preds = []
    for j in range(preds.shape[0]):
        temp = []
        for k in preds[j, :]:
            k = k - 1
            if k == -1:
                temp.append("§")
            else:
                p = encoder.inverse_transform([k])[0]
                temp.append(p)
        tp = "".join(temp).replace("§", "")
        cap_preds.append(remove_duplicates(tp))
    return cap_preds

In [24]:
def remove_duplicates(x):
    if len(x) < 2:
        return x
    fin = ""
    for j in x:
        if fin == "":
            fin = j
        else:
            if j == fin[-1]:
                continue
            else:
                fin = fin + j
    return fin

In [31]:
run_training()

  1%|          | 1/117 [00:00<01:01,  1.89it/s]

torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])


  3%|▎         | 4/117 [00:00<00:43,  2.62it/s]

torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size

  9%|▊         | 10/117 [00:00<00:22,  4.71it/s]

...

torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size

 71%|███████   | 83/117 [00:03<00:01, 24.41it/s]

torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.Size([8, 128, 75, 297])
torch.Size([8, 128, 37, 148])
torch.Size([8, 64, 37, 145])
torch.Size([8, 64, 18, 72])
torch.Size([8, 72, 64, 18])
torch.Size([8, 72, 1152])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 64])
torch.Size([8, 72, 20])
torch.Size([72, 8, 20])
torch.S

 74%|███████▎  | 86/117 [00:03<00:01, 22.31it/s]


KeyboardInterrupt: 